In [1]:
spark

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [3]:
images_folders = ['/CA1/Images/Ireland/','/CA1/Images/Honduran/']

In [4]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [5]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@1bb6f8a0

In [6]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [7]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [8]:
pandasImagesDF = imageDf.toPandas()

In [9]:
def processImage(data, target_size=(225,225)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(new_image).reshape([target_size[0],target_size[1],3])
        return array

In [10]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [11]:
# pandasImagesDF

In [12]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [13]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    180 non-null    object
 1   Label   180 non-null    object
 2   Data    180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [14]:
#import matplotlib.pyplot as plt

In [15]:
#np.array(pandasImagesDF["Data"][9]).astype("float32").shape

In [16]:
# # Determine the dimensions of the image
# width =  225
# height =  225

# # Convert the byte array to a numpy array of uint8 data type
# image_data = np.array(pandasImagesDF["Data"][0], dtype=np.uint8)
# # image_data.reshape(-1,1)
# # Reshape the array to match the dimensions of the image
# # image_data = image_data.reshape((height, width))

# # Create an image from the byte array
# image = Image.fromarray(image_data)

# # Display the image
# image.show()

Importing from Keras functionality necessary to implement CNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score

2024-03-21 19:14:14.573999: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:14:14.598218: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 19:14:14.732923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 19:14:15.481296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


For Now there is only 20 photos per Category, the training size will be 28 and test 12, This is for testing. After this I'll get more images to improve the CNN

In [18]:
TestSize = int(len(pandasImagesDF)*0.3)

In [19]:
X_train = pandasImagesDF.iloc[:-TestSize,2].values
y_train = pandasImagesDF.iloc[:-TestSize,1].values
X_test = pandasImagesDF.iloc[-TestSize:,2].values
y_test = pandasImagesDF.iloc[-TestSize:,1].values


In [20]:
from sklearn.preprocessing import OneHotEncoder

In [21]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [22]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [23]:
X_train_reshaped = []
X_test_reshaped = []

for x in X_train:
    X_train_reshaped.append(np.array(x)[0])

for x in X_test:
    X_test_reshaped.append(np.array(x)[0])
    

In [24]:

# X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
# y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1],1))

Because the pixels are from 0 to 255, We have to normalize the pixels

In [25]:
X_train_reshaped = np.array(X_train_reshaped)/255
X_test_reshaped= np.array(X_test_reshaped)/255

In [26]:
X_train_reshaped.shape,X_test_reshaped.shape

((126, 225, 225, 3), (54, 225, 225, 3))

In [27]:
y_train.shape, y_test.shape

((126, 2), (54, 2))

In [66]:
model = Sequential()
model.add(Input(shape=(225, 225, 3)))  # 225x225 RGB images
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(MaxPool2D(3))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))

model.add(Flatten())


model.add(Dense(128, activation="relu"))
model.add(Dense(2,activation="sigmoid"))

model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [81]:
model.fit(X_train_reshaped, y_train, batch_size=50, epochs=35, verbose=1, validation_data=(X_test_reshaped, y_test))

Epoch 1/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.8292 - loss: 0.3550 - val_accuracy: 0.5000 - val_loss: 1.0229
Epoch 2/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.8796 - loss: 0.3586 - val_accuracy: 0.5556 - val_loss: 0.8105
Epoch 3/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.7964 - loss: 0.4702 - val_accuracy: 0.5000 - val_loss: 0.6646
Epoch 4/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9149 - loss: 0.3484 - val_accuracy: 0.5185 - val_loss: 1.1314
Epoch 5/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.8844 - loss: 0.2939 - val_accuracy: 0.5741 - val_loss: 0.7923
Epoch 6/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9228 - loss: 0.2509 - val_accuracy: 0.5370 - val_loss: 0.9500
Epoch 7/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9612 - loss: 0.2196 - val_accuracy: 0.5185 - val_loss: 1.3593
Epoch 8/35
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9372 - loss: 0.1743 - val_accuracy: 0.5741 - val_loss: 1.4060
Epoch 9/35
3/3 ━

In [48]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 223, 223, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 221, 221, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 71, 71, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 69, 69, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 34, 34, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 505,256 (1.93 MB)

 Trainable params: 168,418 (657.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 336,838 (1.28 MB)

In [82]:
pred = model.predict(X_test_reshaped[:25])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


In [83]:
def comparePredictions(predictionsValues, expectedValues):
    correct = 0
    for x in range(len(predictionsValues)):
        if predictionsValues[x] == expectedValues[x]:
            correct+=1
    
    print(f'Correct predictions {correct}/{len(predictionsValues)}. Percent {correct/len(predictionsValues)}')

In [84]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))

Correct predictions 9/25. Percent 0.36
